Инсубординация(вроде), которую увидела глазами
- Et si je m'explose la tête sur le bitume?
- Si possible.
- Si vous saviez mon téléphone !
- mais si tu fais une erreur. tu mourras en mer.


ЗАМЕНИТЬ ... НА , или лучше на ничего

# Make  DataFrame

In [2]:
import os
import re
import nltk
import pandas as pd
import codecs
from nltk.tokenize import sent_tokenize
from tqdm.auto import tqdm
nltk.download('punkt')

!pip install stanza
import stanza
stanza.download('fr') # download French model
nlp_stanza = stanza.Pipeline('fr') # initialize French neural pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 286kB 4.4MB/s 


2021-04-27 20:51:09 INFO: Downloading default packages for language: fr (French)...
2021-04-27 20:53:13 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-04-27 20:53:13 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | wikiner |

2021-04-27 20:53:13 INFO: Use device: cpu
2021-04-27 20:53:13 INFO: Loading: tokenize
2021-04-27 20:53:13 INFO: Loading: mwt
2021-04-27 20:53:13 INFO: Loading: pos
2021-04-27 20:53:14 INFO: Loading: lemma
2021-04-27 20:53:14 INFO: Loading: depparse
2021-04-27 20:53:14 INFO: Loading: ner
2021-04-27 20:53:16 INFO: Done loading processors!


In [3]:
!pip install --upgrade spicy
!python -m spacy download fr_core_news_md
import fr_core_news_md
nlp = fr_core_news_md.load()

     |████████████████████████████████| 88.6MB 1.2MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp37-none-any.whl size=90338490 sha256=9fd67bb50e719d2e16562753ca4c1426ffce56a9b0afc6723608afd75ea17110
  Stored in directory: /tmp/pip-ephem-wheel-cache-d0l2a6ea/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [4]:
!pip install benepar
from nltk import Tree
import benepar
benepar.download('benepar_fr2')
import warnings
warnings.filterwarnings("ignore")
import spacy
from benepar.spacy_plugin import BeneparComponent
nlp.add_pipe(BeneparComponent('benepar_fr2'))

     |████████████████████████████████| 3.3MB 8.5MB/s 
     |████████████████████████████████| 2.1MB 39.2MB/s 
     |████████████████████████████████| 1.2MB 43.8MB/s 
     |████████████████████████████████| 901kB 51.6MB/s 
  Created wheel for benepar: filename=benepar-0.2.0-cp37-none-any.whl size=37647 sha256=5f88f339b3f64bfaeb922ff11c2aa16603c35828c4e16baa705f538fd26d5ba6
  Stored in directory: /root/.cache/pip/wheels/dd/90/28/513063023646df7774be9401c440a5f13b48bdbab15a67fc42
Successfully built benepar
[nltk_data] Downloading package benepar_fr2 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_fr2.zip.


In [4]:
list_of_paths = []

for root, dirs, files in os.walk("/content/drive/MyDrive/ВКР/proga/DATA/friends"):
    for name in files:
        path = str(root + '/' +  name)
        list_of_paths.append(path)
        #print(os.path.join(root, name))

len(list_of_paths)

145

In [15]:
print(check_unicode('''/content/drive/MyDrive/ВКР/proga/DATA/friends/19 - The One  Where Eddie Won't Go.srt'''))

utf-8


In [16]:
def check_unicode(path):
  encodings = ['utf-8', 'windows-1252', 'windows-1250'] # add more
  for e in encodings:
    try:
      fh = codecs.open(path, 'r', encoding=e)
      fh.readlines()
      fh.seek(0)
    except UnicodeDecodeError:
      pass
        #print('got unicode error with %s , trying different encoding' % e)
    else:
      return e 
      break 

print(check_unicode(list_of_paths[1]))

windows-1252


In [23]:
def make_dataframe(list_of_path):
  data = pd.DataFrame()
  sentences = []
  for path in list_of_path:
    text = []
    encoding = check_unicode(path)
    with open(path,'r', encoding=encoding) as f: 
      lines = f.readlines()
    for x in lines:
      if not re.search(r'[a-zA-Z]', x):
        pass
      else:
        x = re.sub(r'\d\d:\d\d:\d\d,\d\d\d --> \d\d:\d\d:\d\d,\d\d\d  X\d:\d\d\d X\d:\d\d\d Y\d:\d\d\d Y\d:\d\d\d', r'', x)
        if len(x) > 1:
          text.append(x.replace('\n',''))
    sentences = sentences +  sent_tokenize(' '.join(text))
  data['sentences'] = sentences
  return data

sentences = make_dataframe(list_of_path = list_of_paths)

In [24]:
len(sentences)

49812

In [28]:
dependence = []
for i, text in tqdm(enumerate(sentences.sentences)):
  doc = nlp_stanza(text)
  list_doc = doc.to_dict()
  dependence.append(list_doc)
sentences['dependence_structure'] = dependence 

sentences.head()

,sentences,dependence_structure
0,"Vous pigez pas... que pour nous, les baisers o...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up..."
1,Ben voyons !,"[[{'id': 1, 'text': 'Ben', 'lemma': 'Ben', 'up..."
2,Sérieux ?,"[[{'id': 1, 'text': 'Sérieux', 'lemma': 'série..."
3,Tout se lit déjà dans le premier baiser.,"[[{'id': 1, 'text': 'Tout', 'lemma': 'tout', '..."
4,Absolument !,"[[{'id': 1, 'text': 'Absolument', 'lemma': 'ab..."


In [12]:
sentences.to_csv('friends_dependence.csv', index=False)

In [13]:
sentences = pd.read_csv('/content/drive/MyDrive/ВКР/proga/DATA/friends_dependences.csv')

In [9]:
# находим предложения, где есть починительный союз
idxs_sconj = []
for i, text in enumerate(sentences.dependence_structure):
  for sent in eval(text):
    for word in sent:
      try:
        if word['upos'] == 'SCONJ':
          if i not in idxs_sconj:
            idxs_sconj.append(i)
      except:
        continue

len(idxs_sconj)

5500

In [10]:
sconj = []
for i in range(len(sentences)):
  if i not in idxs_sconj:
    sconj.append(0)
  else:
    sconj.append(1)

sentences['SCONJ'] = sconj

In [12]:
si = []
que = []
comme = []
lorsque = []
quand = []
sinon = []
puisque = []

for i in idxs_sconj:
  for sent in eval(sentences.dependence_structure[i]):
    for word in sent:
      try:
        if word['upos'] == 'SCONJ':
          if word['lemma'] == 'si':
            si.append(i)
          if word['lemma'] == 'que':
            que.append(i)
          if word['lemma'] == 'comme':
            comme.append(i)
          if word['lemma'] == 'lorsque':
            lorsque.append(i)
          if word['lemma'] == 'quand':
            quand.append(i)
          if word['lemma'] == 'sinon':
            sinon.append(i)
          if word['lemma'] == 'puisque':
            puisque.append(i)
      except:
        continue

print('si-----',len(si))
print('que-----', len(que))
print('comme-----',len(comme))
print('lorsque-----',len(lorsque))
print('quand-----',len(quand))
print('sinon-----',len(sinon))
print('puisque-----',len(puisque))

si----- 1393
que----- 3938
comme----- 236
lorsque----- 2
quand----- 571
sinon----- 16
puisque----- 25


In [52]:
for x in sentences.loc[sentences['SCONJ'] == 1].sentences:
  print(x)

Выходные данные были обрезаны до нескольких последних строк (5000).
Ils t'ont amené pour que je te voie.
Chaque fois qu'on sort avec des patientes... Détends-toi.
Je sais que c'est assez égoïste... mais je suis comme ça !
Au fait, quand j'étais au lycée, j'étais une vraie vache.
Mais il y a une chose... que je voulais te dire.
Je vais voir si elle est là.
C'est désagréable, quand les gens te font faux bon !
Au restau, ils m'ont dit qu'elle était occupée.
Je peux pas croire qu'elle me largue !
Parce que... je me suis encore trompée de nom.
Au pire des cas... imagine que tu ne te sentes jamais père.
Imagine que ton fils ne se sente jamais ton fils.
Imagine que toutes ses relations en soient affectées.
Qu'est-ce qu'il a, le singe ?
Il se touche quand personne ne le regarde.
Parce que je suis copain avec Phoebe ?
Si c'était ça, tu arrêterais de la voir ?
Je ne sais pas si c'est parce qu'on se sépare... mais tu n'as jamais été si belle.
Qu'est-ce que c'est ?
Ce serait différent si j'étais a

In [1]:
# показывает дерево предложения
def make_tree(text):
  doc = nlp(text)
  sents = list(doc.sents)
  for sent in sents:
    print(sent._.parse_string)
    result = Tree.fromstring(str(sent._.parse_string)).pretty_print()
  pass

# строит структуру составляющих предложения
def make_structure(text):
    doc = nlp(text)
    sents = list(doc.sents)
    result = [sent._.parse_string for sent in sents]
    return result

# парсим структуру составляющих + берем только уровень 1
def parenthetic(string):
    result = []
    stack = []
    for i, c in enumerate(string):
        if c == '(':
            stack.append(i)
        elif c == ')' and stack:
            start = stack.pop()
            if len(stack) == 1:
              result.append(string[start + 1: i].split(' ')[0])
    return result 

def dep_struct(text):
  doc = nlp_stanza(text)
  list_doc = doc.to_dict()
  return list_doc

In [14]:
sentences

,sentences,dependence_structure,SCONJ
0,"Vous pigez pas... que pour nous, les baisers o...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",1
1,Ben voyons !,"[[{'id': 1, 'text': 'Ben', 'lemma': 'Ben', 'up...",0
2,Sérieux ?,"[[{'id': 1, 'text': 'Sérieux', 'lemma': 'série...",0
3,Tout se lit déjà dans le premier baiser.,"[[{'id': 1, 'text': 'Tout', 'lemma': 'tout', '...",0
4,Absolument !,"[[{'id': 1, 'text': 'Absolument', 'lemma': 'ab...",0
...,...,...,...
49807,Ma titine est enfin libre !,"[[{'id': 1, 'text': 'Ma', 'lemma': 'son', 'upo...",0
49808,Démarre !,"[[{'id': 1, 'text': 'Démarre', 'lemma': 'démar...",0
49809,On est pas sexy ?,"[[{'id': 1, 'text': 'On', 'lemma': 'on', 'upos...",0
49810,Tu veux acheter une caisse ?,"[[{'id': 1, 'text': 'Tu', 'lemma': 'il', 'upos...",0


In [24]:
trees = []
for x in tqdm(sentences.loc[sentences['SCONJ'] == 1].sentences):
  print(len(make_structure(x)))

sentences['tree_structure'] = trees

2
1
1
1
1
1
2
1
1
1


In [19]:
trees = []
for text in tqdm(sentences.sentences):
  trees.append(make_structure(text))

sentences['tree_structure'] = trees 


In [20]:
sentences.to_csv('friends_dependence.csv', index=False)

In [21]:
sentences.to_csv('/content/drive/MyDrive/ВКР/proga/DATA/friends_dependences_new.csv', index=False)

# Look at  ready DataFrame

это инсубординация?
- Si je le crois ? 
- Faut que je la mette bien en vue.
- S'il te plaît. - не считать?так как это устоявшаяся штука

In [5]:
import pandas as pd
sentences = pd.read_csv('/content/drive/MyDrive/ВКР/proga/DATA/friends_dependences_new.csv')
sentences.head(3)

,sentences,dependence_structure,SCONJ,tree_structure
0,"Vous pigez pas... que pour nous, les baisers o...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",1,['(SENT (VN (CLS Vous) (V pigez)) (ADV pas) (P...
1,Ben voyons !,"[[{'id': 1, 'text': 'Ben', 'lemma': 'Ben', 'up...",0,['(SENT (ADV Ben) (VN (VIMP voyons)) (PONCT !))']
2,Sérieux ?,"[[{'id': 1, 'text': 'Sérieux', 'lemma': 'série...",0,['(SENT (AP (ADJ Sérieux)) (PONCT ?))']


In [6]:
# Добавим еще одну колонку - закодируем структуру составляющих для обучения
# парсим структуру составляющих
def parenthetic_full(string):
    result = []
    stack = []
    for i, c in enumerate(string):
        if c == '(':
            stack.append(i)
        elif c == ')' and stack:
            start = stack.pop()
            result.append((len(stack),string[start + 1: i].split(' ')[0]))
    return sorted(result)

In [7]:
tags = [] # 57 тегов
for struct in sentences['tree_structure']:
  for s in eval(struct):
    for x in parenthetic_full(s):
      if x[1] not in tags:
        tags.append(x[1])
tags.append('PAD')

In [8]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
enc = OrdinalEncoder()
X = np.array(tags).reshape(-1, 1)
enc.fit(X)
enc.transform([['SENT']])

array([[41.]])

In [22]:
def func(s, n):
  result = []
  for x in parenthetic_full(s):
    try:
      if x[0] == n:
        result.append(int(enc.transform([[x[1]]])))
    except:
      result.append(int(enc.transform([['PAD']])))
  return np.array(result)

for struct in list(sentences['tree_structure'])[:10]:
  for s in eval(struct):
    print(func(s,0))

[41]
[41]
[41]
[41]
[41]
[41]
[41]
[41]
[41]
[41]
[41]


In [23]:
# попробуем разделить уровни составляющих по разным колонкам, максимальный уровень - 4
zeros = []
ones = []
twos = []
threes = []
fours = []
for struct in list(sentences['tree_structure']):
  for s in eval(struct):  
    zeros.append(func(s,0))
    ones.append(func(s,1))
    twos.append(func(s,2))
    threes.append(func(s,3))
    fours.append(func(s,4))
    
sentences['zeros'] = zeros
sentences['ones']= ones
sentences['twos']= twos
sentences['threes'] = threes
sentences['fours'] = fours    

sentences.head(3)

ValueError: ignored

In [61]:
l = [5, 6]
k = ['yes'  if x==5 else 'no' for x in l]
k

['yes', 'no']

In [72]:
struct_decode = [] 
for struct in sentences['tree_structure']:
  code = []
  for s in eval(struct):
    for x in parenthetic_full(s):
      x_new = enc.transform([[x[1]]])   
      l = [int(x[0]), int(x_new[0][0])]
      code.append(l)
  code = np.array([np.array(xi, dtype=object) for xi in code], dtype=object)
  struct_decode.append(np.array(code))
sentences['struct_decode'] = struct_decode
sentences.head()

,sentences,dependence_structure,SCONJ,tree_structure,struct_decode,target
0,"Vous pigez pas... que pour nous, les baisers o...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",1,['(SENT (VN (CLS Vous) (V pigez)) (ADV pas) (P...,"[[0, 40], [1, 3], [1, 33], [1, 49], [2, 12], [...",0
1,Ben voyons !,"[[{'id': 1, 'text': 'Ben', 'lemma': 'Ben', 'up...",0,['(SENT (ADV Ben) (VN (VIMP voyons)) (PONCT !))'],"[[0, 40], [1, 3], [1, 33], [1, 49], [2, 46]]",0
2,Sérieux ?,"[[{'id': 1, 'text': 'Sérieux', 'lemma': 'série...",0,['(SENT (AP (ADJ Sérieux)) (PONCT ?))'],"[[0, 40], [1, 6], [1, 33], [2, 0]]",0
3,Tout se lit déjà dans le premier baiser.,"[[{'id': 1, 'text': 'Tout', 'lemma': 'tout', '...",0,['(SENT (NP (PRO Tout)) (VN (CLR se) (V lit)) ...,"[[0, 40], [1, 3], [1, 25], [1, 33], [1, 34], [...",0
4,Absolument !,"[[{'id': 1, 'text': 'Absolument', 'lemma': 'ab...",0,['(SENT (ADV Absolument) (PONCT !))'],"[[0, 40], [1, 3], [1, 33]]",0


In [9]:
list(sentences['struct_decode'])[0].shape

(24, 2)

In [11]:
for x in list(sentences['struct_decode'])[:2]:
  print(x.shape)

(24, 2)
(5, 2)


In [7]:
for i,row in sentences.iterrows():
  if row['SCONJ'] == 1:
    trees = [parenthetic(sent) for sent in eval(row.tree_structure)]
    print(row.sentences, '-------', trees)


Выходные данные были обрезаны до нескольких последних строк (5000).
Ils t'ont amené pour que je te voie. ------- [['VN', 'Ssub', 'PONCT']]
Chaque fois qu'on sort avec des patientes... Détends-toi. ------- [['Ssub', 'PONCT'], ['VN', 'PONCT']]
Je sais que c'est assez égoïste... mais je suis comme ça ! ------- [['VN', 'Ssub', 'PONCT', 'COORD', 'PONCT']]
Au fait, quand j'étais au lycée, j'étais une vraie vache. ------- [['ADV+', 'PONCT', 'Ssub', 'PONCT', 'VN', 'NP', 'PONCT']]
Mais il y a une chose... que je voulais te dire. ------- [['COORD', 'PONCT']]
Je vais voir si elle est là. ------- [['VN', 'VPinf', 'PONCT']]
C'est désagréable, quand les gens te font faux bon ! ------- [['VN', 'AP', 'PONCT', 'Ssub', 'PONCT']]
Au restau, ils m'ont dit qu'elle était occupée. ------- [['PP', 'PONCT', 'VN', 'Ssub', 'PONCT']]
Je peux pas croire qu'elle me largue ! ------- [['VN', 'ADV', 'VPinf', 'Ssub', 'PONCT']]
Parce que... je me suis encore trompée de nom. ------- [['Ssub', 'PONCT']]
Au pire des cas...

In [22]:
from tqdm.auto import tqdm
insub_structures = [['Ssub', 'PONCT'],['Ssub'],['ADV', 'ADV', 'VN', 'PONCT']] #['COORD', 'PONCT'], ['COORD'],
insub_list = []
for i, row in  tqdm(sentences.loc[sentences['SCONJ'] == 1].iterrows()):
  for sent in eval(row.tree_structure):
    if parenthetic(sent) in insub_structures:
      insub_list.append(i)
len(insub_list)

711

In [64]:
text = '''Et si l’on vous découvre et qu’on vous force a partir?'''
doc = nlp(text)
struct = list(doc.sents)[0]
print(struct._.parse_string)
print(struct._.labels)
list(struct._.constituents)

#parenthetic(make_structure(text)[0])
#make_tree(text)

# очень плохо и непонятно,  что с этим делать? кажется, нужно смотреть на грамматику:
# Qu'il sorte! = ['ADV', 'ADV', 'VN', 'PONCT']
# Qu'il ne replique pas! = ['ADV', 'ADV', 'VN', 'ADV', 'PONCT', 'PONCT']
# Si je pouvais juste arreter a ce moment la  = ['CS', 'VN', 'ADV', 'VPinf']

(SENT (COORD (CC Et) (CS si) (Sint (VN (DET l’) (CLS on) (CLO vous) (V découvre)) (COORD (CC et) (Ssub (CS qu’) (Sint (VN (CLS on) (CLO vous) (V force)) (PP (P a) (VPinf (VN (VINF partir))))))))) (PONCT ?))
('SENT',)


[Et si l’on vous découvre et qu’on vous force a partir?,
 Et si l’on vous découvre et qu’on vous force a partir,
 Et,
 si,
 l’on vous découvre et qu’on vous force a partir,
 l’on vous découvre,
 l’,
 on,
 vous,
 découvre,
 et qu’on vous force a partir,
 et,
 qu’on vous force a partir,
 qu’,
 on vous force a partir,
 on vous force,
 on,
 vous,
 force,
 a partir,
 a,
 partir,
 ?]

In [13]:
insub_list = [x for x in insub_list if '...' not in sentences.sentences.loc[x]]
len(insub_list)

452

In [14]:
for x in insub_list:
  print(sentences.sentences.loc[x])

Parce que mes parents lui donnent le bon Dieu sans confession !
Comme si je ne redoutais pas assez de devoir la lui rendre demain.
Que je puisse souffler.
Si c'est un garcon.
Parce qu'il est un peu analytique ?
A moins que j'y aille, et que je me lève tôt pour les remettre.
Qu'on en achète une ensemble ?
Si l'un de nous déménage ?
Comme tu veux.
Si tu le dis !
Si on mettait le Touti sur le palier ?
Marre qu'on m'appelle "Excusez-moi" !
S'il te plaît !
Parce que j'ai quatre six !
Quand Harry rencontre Sally !
Parce que je pourrais très bien décrocher un Brian !
Sauf si on prend une autre robe.
Qu'elle te rendait dingue, à obséder sur le moindre petit détail.
Qu'il se retourne et dise : "Ca me plaît, je te veux" ?
Si on pouvait faire quelque chose !
Parce que sa mère risque de le faire.
Comme s'il allait t'en envoyer une !
Parce que c'est toi.
Parce que si je pars, je dois rompre avec toi.
Faut qu'on m'embrasse !
Que quelqu'un m'embrasse !
Que tu ouvres la porte.
Faut que je voie ca !
Po

In [23]:
insub_idxs = []
for i in range(len(sentences)):
  if i not in insub_list:
    insub_idxs.append(0)
  else:
    insub_idxs.append(1)

sentences['target'] = insub_idxs 
sentences.head(3)

,sentences,dependence_structure,SCONJ,tree_structure,struct_decode,target
0,"Vous pigez pas... que pour nous, les baisers o...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",1,['(SENT (VN (CLS Vous) (V pigez)) (ADV pas) (P...,"[[0, 40], [1, 3], [1, 33], [1, 49], [2, 12], [...",0
1,Ben voyons !,"[[{'id': 1, 'text': 'Ben', 'lemma': 'Ben', 'up...",0,['(SENT (ADV Ben) (VN (VIMP voyons)) (PONCT !))'],"[[0, 40], [1, 3], [1, 33], [1, 49], [2, 46]]",0
2,Sérieux ?,"[[{'id': 1, 'text': 'Sérieux', 'lemma': 'série...",0,['(SENT (AP (ADJ Sérieux)) (PONCT ?))'],"[[0, 40], [1, 6], [1, 33], [2, 0]]",0


# Попробуем что-нибудь обучить на закодированной структуре составляющих

In [51]:
sentences.head(2)

,sentences,dependence_structure,SCONJ,tree_structure,struct_decode,target
0,"Vous pigez pas... que pour nous, les baisers o...","[[{'id': 1, 'text': 'Vous', 'lemma': 'il', 'up...",1,['(SENT (VN (CLS Vous) (V pigez)) (ADV pas) (P...,"[[0, 40], [1, 3], [1, 33], [1, 49], [2, 12], [...",0
1,Ben voyons !,"[[{'id': 1, 'text': 'Ben', 'lemma': 'Ben', 'up...",0,['(SENT (ADV Ben) (VN (VIMP voyons)) (PONCT !))'],"[[0, 40], [1, 3], [1, 33], [1, 49], [2, 46]]",0


In [82]:
from sklearn.model_selection import train_test_split
X = sentences[['struct_decode']] #'SCONJ',
y = sentences['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y, shuffle=True)

In [58]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
def scores(model):
    print('Accuracy: ', round(model.score(X_test, y_test),4))
    y_score = model.decision_function(X_test)
    average_precision = average_precision_score(y_test, y_score)
    print(f'Precision: {average_precision}')
    y_pred = model.predict(X_test)
    recall = recall_score(y_test, y_pred)
    print(f'Recall: {recall}')
    f1 = f1_score(y_test, y_pred)
    print(f'F1 score: {round(f1,4)}')

In [74]:
X_train.head(3)

,struct_decode
30753,"[[0, 40], [1, 3], [1, 25], [1, 33], [1, 34], [..."
729,"[[0, 40], [1, 13], [1, 25], [1, 33], [1, 43], ..."
4231,"[[0, 40], [1, 33], [1, 49], [2, 10], [2, 33], ..."


In [75]:
type(X_train.struct_decode[0]), type(X_train.struct_decode[0][0])

(numpy.ndarray, numpy.ndarray)

In [76]:
X_train.struct_decode[0]

array([[0, 40],
       [1, 3],
       [1, 33],
       [1, 49],
       [2, 12],
       [2, 44],
       [0, 40],
       [1, 25],
       [1, 25],
       [1, 33],
       [1, 33],
       [1, 43],
       [1, 49],
       [2, 0],
       [2, 14],
       [2, 16],
       [2, 16],
       [2, 23],
       [2, 23],
       [2, 34],
       [2, 44],
       [3, 25],
       [3, 28],
       [4, 36]], dtype=object)

In [86]:
print(type(list(X_train.struct_decode)[0]))
list(X_train.struct_decode)[0].shape

<class 'numpy.ndarray'>


(20, 2)

In [81]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

ValueError: ignored